In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Get the dataset

In [ ]:
import urllib.request
from zipfile import ZipFile

dataDir = 'dataset'

datasetUrl = 'https://archive.ics.uci.edu/static/public/45/heart+disease.zip'
datasetZipFile = os.path.join(dataDir, 'heart-disease.zip')
if not os.path.isdir(dataDir):
    os.makedirs(dataDir)
urllib.request.urlretrieve(datasetUrl, datasetZipFile)

with ZipFile(datasetZipFile, 'r') as f:
    f.extractall(dataDir)

### Notes
Need to determine whether it makes sense for columns with discrete values to be represented using ordinal encoding or one-hot encoding.
If the values are based on binning a continuous variable, then the values are ordered as they should be.
If the values are arbitrarily assigned based on category, then they should be transformed using something like one-hot encoding.

Column descriptions taken from https://archive.ics.uci.edu/dataset/45/heart+disease:

age: age in years

sex: sex (1 = male; 0 = female)

cp: chest pain type

        -- Value 1: typical angina

        -- Value 2: atypical angina

        -- Value 3: non-anginal pain

        -- Value 4: asymptomatic

trestbps: resting blood pressure (in mm Hg on admission to the hospital)

chol: serum cholestoral in mg/dl

fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)

restecg: resting electrocardiographic results

        -- Value 0: normal

        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

thalach: maximum heart rate achieved

exang: exercise induced angina (1 = yes; 0 = no)

oldpeak = ST depression induced by exercise relative to rest

slope: the slope of the peak exercise ST segment

        -- Value 1: upsloping

        -- Value 2: flat

        -- Value 3: downsloping

ca: number of major vessels (0-3) colored by flourosopy

thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

num: diagnosis of heart disease (target)

## Load the dataset and explore

In [ ]:
clevelandDataFile = os.path.join(dataDir, 'processed.cleveland.data')

In [ ]:
columnNames = [  # obtained from https://archive.ics.uci.edu/dataset/45/heart+disease
    'age',
    'sex',
    'cp',
    'trestbps',
    'chol',
    'fbs',
    'restecg',
    'thalach',
    'exang',
    'oldpeak',
    'slope',
    'ca',
    'thal',
    'num'
]
clevelandData = pd.read_csv(
    clevelandDataFile,
    header=None,
    names=columnNames
)

In [ ]:
clevelandData.head()

Columns 11 and 12 (`'ca'` and `'thal'`) are of Dtype `object`, which likely means that there are missing values.

In [ ]:
clevelandData.info()

Looking at histograms of the column values, one can make the following observations:
- sex, cp, fbs, restecg, exang, slope, and num are discrete variables
- trestbps, chol, and oldpeak might have outliers
- trestbps appears to have much larger counts for a small set of discrete values, possibly indicating binning for some subset of the data
- age could be multi-model
- oldpeak has a much larger number of 0 values comapared to the others
- restecg has a very small number of entries with value 1 compared to 0 and 2

In [ ]:
clevelandData.hist(bins=30, figsize=(15,15))
plt.show()

Looking at the counts for the unique values in columns `'ca'` and `'thal'`, we confirm that these columns appear to have missing values indicated by `'?'`.

In [ ]:
clevelandData['ca'].value_counts()

In [ ]:
clevelandData['thal'].value_counts()

## Data cleaning

#### Replace `'?'` with NaNs

In [ ]:
clevelandData['ca'] = clevelandData['ca'].apply(lambda x: np.nan if x=='?' else np.float64(x))
clevelandData['thal'] = clevelandData['thal'].apply(lambda x: np.nan if x=='?' else np.float64(x))

#### Impute NaNs with the median
Another option would be to drop the rows with NaNs since there are a small number of them).
Question: does using the median make sense here? Would using the mode also make sense?

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(clevelandData)

## Could also transform some columns to one-hot encoding if necessary

Possible candidates for transform:

sex

cp: chest pain type

        -- Value 1: typical angina

        -- Value 2: atypical angina

        -- Value 3: non-anginal pain

        -- Value 4: asymptomatic

restecg: resting electrocardiographic results

        -- Value 0: normal

        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)

        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

slope: the slope of the peak exercise ST segment

        -- Value 1: upsloping

        -- Value 2: flat

        -- Value 3: downsloping

In [ ]:
numerical_cols = [
    'age',
    'trestbps',
    'chol',
    'fbs',
    'thalach',
    'exang',
    'oldpeak',
    'ca',
    'thal'
]
categorical_cols = ['sex', 'cp', 'restecg', 'slope']

## Make target variable binary

Based on the dataset description, any entries with values of the `num` column that are greater than zero should be considered as having heart disease, and entries with values of zero should be considered as not having heart disease.

In [ ]:
clevelandData['target'] = clevelandData['num'].apply(lambda x: 1 if x > 0 else 0)
clevelandData.head()

## Create a test set

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)
for idx_train, idx_test in splitter.split(clevelandData, clevelandData['num']):
    clevelandData_train = clevelandData.loc[idx_train]
    clevelandData_test = clevelandData.loc[idx_test]

In [ ]:
clevelandData_train['num'].value_counts() / clevelandData_train.shape[0]

In [ ]:
clevelandData_test['num'].value_counts() / clevelandData_test.shape[0]

In [ ]:
clevelandData['num'].value_counts() / clevelandData.shape[0]

In [ ]:
for set_ in (clevelandData_train, clevelandData_test):
    set_.drop("num", axis=1, inplace=True)

## Look for correlations between columns

Interestingly, the target variable has very low correlation with respect to cholesterol level!

In [ ]:
corr_matrix = clevelandData_train.corr()

In [ ]:
corr_matrix['target'].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = [
    'target', 'thal', 'exang', 'ca', 'oldpeak', 'cp', 'slope', 'age', 'sex',
    'trestbps', 'restecg', 'thalach'
]
scatter_matrix(
    clevelandData_train[attributes],
    figsize=(12,12)
)
plt.show()

## Build pipelines

In [ ]:
train_y = clevelandData_train['target'].copy().to_numpy()
clevelandData_train_X = clevelandData_train.drop('target', axis=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', OneHotEncoder(), categorical_cols)
])

train_X = full_pipeline.fit_transform(clevelandData_train_X)

In [ ]:
train_X.shape

In [ ]:
train_y.shape

## Train and evaluate models

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=1234)
scores = cross_val_score(tree_clf, train_X, train_y,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=1234)
forest_scores = cross_val_score(forest_clf, train_X, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
svm_clf = SVC(kernel='linear', random_state=1234)
svm_scores = cross_val_score(svm_clf, train_X, train_y,
                                scoring="neg_mean_squared_error", cv=10)
svm_rmse_scores = np.sqrt(-svm_scores)
display_scores(svm_rmse_scores)

In [ ]:
log_reg = LogisticRegression(penalty='l2', random_state=1234)  # L2 regularization is used by default
log_reg_scores = cross_val_score(log_reg, train_X, train_y,
                                scoring="neg_mean_squared_error", cv=10)
log_reg_rmse_scores = np.sqrt(-log_reg_scores)
display_scores(log_reg_rmse_scores)

## Grid search hyperparameters

### Logistic regression

In [ ]:
param_grid = [
    # try 7 combinations of hyperparameters
    {'C': [1/10, 1/4, 1/2, 1, 2, 4, 10],
     'penalty': ['l2', 'l1']},
  ]

log_reg = LogisticRegression(solver='liblinear', random_state=1234)
grid_search = GridSearchCV(log_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_X, train_y)

In [ ]:
print(grid_search.best_params_)
cvres = grid_search.cv_results_
idx = np.argsort(cvres['mean_test_score'])[::-1]
for k in idx:
    print(
        'Test score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_test_score'][k]), np.sqrt(cvres['std_test_score'][k])),
        'Train score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_train_score'][k]), np.sqrt(cvres['std_train_score'][k])),
        cvres['params'][k]
    )

### SVM

In [ ]:
param_grid = [
    {'C': [1/10, 1/4, 1/2, 1, 2, 4, 10],
     'kernel': ['linear', 'rbf']},
  ]

svm_clf = SVC(random_state=1234)
grid_search = GridSearchCV(svm_clf, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_X, train_y)

In [ ]:
print(grid_search.best_params_)
cvres = grid_search.cv_results_
idx = np.argsort(cvres['mean_test_score'])[::-1]
for k in idx:
    print(
        'Test score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_test_score'][k]), np.sqrt(cvres['std_test_score'][k])),
        'Train score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_train_score'][k]), np.sqrt(cvres['std_train_score'][k])),
        cvres['params'][k]
    )

### Random forest

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30, 100],
     'max_features': [2, 4, 6, 8, 12, 14],
     'max_depth': [None, 2, 4, 16, 32]},
  ]

forest_clf = RandomForestClassifier(random_state=1234)
grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(train_X, train_y)

In [ ]:
print(grid_search.best_params_)
cvres = grid_search.cv_results_
idx = np.argsort(cvres['mean_test_score'])[::-1]
for k in idx:
    print(
        'Test score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_test_score'][k]), np.sqrt(cvres['std_test_score'][k])),
        'Train score = %.3f+\-%.3f' % (np.sqrt(-cvres['mean_train_score'][k]), np.sqrt(cvres['std_train_score'][k])),
        cvres['params'][k]
    )

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
sorted(zip(feature_importances, clevelandData_train_X.columns), reverse=True)

## Learning curve

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y, random_state):
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
    for idx_train, idx_val in splitter.split(X, y):
        X_train = X[idx_train]
        X_val = X[idx_val]
        y_train = y[idx_train]
        y_val = y[idx_val]
    train_errors, val_errors = [], []
    for m in range(10, len(X_train) + 1):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    fig, ax = plt.subplots()
    ax.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    ax.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    ax.legend(loc="upper right", fontsize=14)
    ax.set_xlabel("Training set size", fontsize=14)
    ax.set_ylabel("RMSE", fontsize=14)
    return fig, ax

In [ ]:
log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=1234)  # L2 regularization is used by default
fig, ax = plot_learning_curves(log_reg, train_X, train_y, random_state=1234)
plt.show()

In [ ]:
svm_clf = SVC(kernel='linear', C=0.5, random_state=1234)
fig, ax = plot_learning_curves(svm_clf, train_X, train_y, random_state=1234)
plt.show()

In [ ]:
forest_clf = RandomForestClassifier(max_depth=2, max_features=6, n_estimators=10, random_state=1234)
fig, ax = plot_learning_curves(forest_clf, train_X, train_y, random_state=1234)
plt.show()

## Confusion matrix

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score

log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=1234)
y_train_pred = cross_val_predict(log_reg, train_X, train_y, cv=10)
conf_mat = confusion_matrix(train_y, y_train_pred)
print(conf_mat)

TP, FP, FN, TN = conf_mat[0,0], conf_mat[0,1], conf_mat[1,0], conf_mat[1,1]
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
acc = (TP + TN) / (TP + TN + FP + FN)
print('acc = %.3f' % acc)
print('precision = %.3f' % precision)
print('recall = %.3f' % recall)
print('f1 = %.3f' % f1_score(train_y, y_train_pred))

In [ ]:
svm_clf = SVC(kernel='linear', C=0.5, random_state=1234)
y_train_pred = cross_val_predict(svm_clf, train_X, train_y, cv=10)
conf_mat = confusion_matrix(train_y, y_train_pred)
print(conf_mat)

TP, FP, FN, TN = conf_mat[0,0], conf_mat[0,1], conf_mat[1,0], conf_mat[1,1]
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
acc = (TP + TN) / (TP + TN + FP + FN)
print('acc = %.3f' % acc)
print('precision = %.3f' % precision)
print('recall = %.3f' % recall)
print('f1 = %.3f' % f1_score(train_y, y_train_pred))

In [ ]:
forest_clf = RandomForestClassifier(max_depth=2, max_features=6, n_estimators=10, random_state=1234)
y_train_pred = cross_val_predict(forest_clf, train_X, train_y, cv=10)
conf_mat = confusion_matrix(train_y, y_train_pred)
print(conf_mat)

TP, FP, FN, TN = conf_mat[0,0], conf_mat[0,1], conf_mat[1,0], conf_mat[1,1]
TP, FP, FN, TN = conf_mat[0,0], conf_mat[0,1], conf_mat[1,0], conf_mat[1,1]
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
precision = TP / (TP + FP)
acc = (TP + TN) / (TP + TN + FP + FN)
print('acc = %.3f' % acc)
print('precision = %.3f' % precision)
print('recall = %.3f' % recall)
print('f1 = %.3f' % f1_score(train_y, y_train_pred))

## Precision vs. recall

In [ ]:
from sklearn.metrics import precision_recall_curve

def plot_precision_vs_recall(precisions, recalls, ax, colorAndProps):
    ax.plot(recalls, precisions, colorAndProps, linewidth=2)
    ax.set_xlabel("Recall", fontsize=16)
    ax.set_ylabel("Precision", fontsize=16)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.grid(True)

In [ ]:
fig, ax = plt.subplots()

# logistic regression
log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=1234)
y_train_scores = cross_val_predict(log_reg, train_X, train_y, cv=10,
                             method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(train_y, y_train_scores)
plot_precision_vs_recall(precisions, recalls, ax, 'r-')

# SVM
svm_clf = SVC(kernel='linear', C=0.5, random_state=1234)
y_train_scores = cross_val_predict(svm_clf, train_X, train_y, cv=10,
                             method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(train_y, y_train_scores)
plot_precision_vs_recall(precisions, recalls, ax, 'b-')

# random forest
forest_clf = RandomForestClassifier(max_depth=2, max_features=6, n_estimators=10, random_state=1234)
y_train_scores = cross_val_predict(forest_clf, train_X, train_y, cv=10,
                             method="predict_proba")
precisions, recalls, thresholds = precision_recall_curve(train_y, y_train_scores[:,1])
plot_precision_vs_recall(precisions, recalls, ax, 'k-')

ax.legend(['Log reg', 'SVM', 'RF'])
plt.show()

# Dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
train_X_pca = pca.fit_transform(train_X)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.cumsum(pca.explained_variance_ratio_), 'o')
plt.show()

In [ ]:
log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=1234)  # L2 regularization is used by default
log_reg_scores = cross_val_score(log_reg, train_X_pca[:,:10], train_y,
                                scoring="neg_mean_squared_error", cv=5)
log_reg_rmse_scores = np.sqrt(-log_reg_scores)
display_scores(log_reg_rmse_scores)

In [ ]:
from sklearn.manifold import TSNE
train_X_tsne = TSNE(
    n_components=2,
    learning_rate='auto',
    init='pca',
    perplexity=50
).fit_transform(train_X)

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    train_X_tsne[train_y==0,0],
    train_X_tsne[train_y==0,1],
    'bo'
)
ax.plot(
    train_X_tsne[train_y==1,0],
    train_X_tsne[train_y==1,1],
    'ro'
)
plt.show()

## Performance on test set

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
test_y = clevelandData_test['target'].copy().to_numpy()
clevelandData_test_X = clevelandData_test.drop('target', axis=1)
test_X = full_pipeline.fit_transform(clevelandData_test_X)

In [ ]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=1234)
forest_clf.fit(train_X, train_y)
print(f'acc = {forest_clf.score(test_X, test_y)}')
print(f'f1 = {f1_score(test_y, forest_clf.predict(test_X))}')

In [ ]:
log_reg = LogisticRegression(penalty='l2', C=0.1, random_state=1234)
log_reg.fit(train_X, train_y)
print(f'acc = {log_reg.score(test_X, test_y)}')
print(f'f1 = {f1_score(test_y, log_reg.predict(test_X))}')

In [ ]:
svm_clf = SVC(kernel='linear', C=0.5, random_state=1234)
svm_clf.fit(train_X, train_y)
print(f'acc = {svm_clf.score(test_X, test_y)}')
print(f'f1 = {f1_score(test_y, svm_clf.predict(test_X))}')